# Website Demo for Infection Risk Calculator with Information Retrieved from Univeristy Systems

**This is the demo for our website application which estimates infection risk based on building information** 

Import necessary packages and files

In [1]:
from jupyter_dash import JupyterDash
import dash


import os

import sys
import dash_core_components as dcc
import dash_html_components as html
import dash
import sys
import os
sys_path = os.path.dirname(os.getcwd())
src_path = sys_path + "/src"

sys.path.insert(1, src_path)
from calculator import *
assumptions = var
src_path = sys_path + "/notebook"

sys.path.insert(1, src_path)

In [2]:
pip install jupyter-dash # Install jupyter-dash for running HTML in Python

Note: you may need to restart the kernel to use updated packages.


'C:\Users\Nicholas' is not recognized as an internal or external command,
operable program or batch file.


Import data

In [3]:
rid_path = 'rm.csv'
#sys_path = os.path.dirname(os.getcwd())


#data_path = sys_path + "/data/"

room_df = pd.read_csv(rid_path)
rooms = room_df.copy()

rooms_id = []
for rid in room_df['Room']:
    rooms_id.append({'label': rid, 'value': rid})
    
rooms.head()

,Room,Area,Height,Windows,Capacity,C19_Occupancy,VAVmin,VAVmax
0,CENTER 101,3428,21.0,NaN,295.0,50.0,3500.0,7000.0
1,CENTER 115,2143,21.0,NaN,196.0,31.0,1290.0,4300.0
2,CENTER 119,2144,21.0,NaN,196.0,30.0,1290.0,4300.0
3,GH 242,2956,NaN,NaN,417.0,50.0,935.0,2325.0
4,LEDDN AUD,1888,NaN,NaN,216.0,30.0,4000.0,12000.0


In [5]:
vav_room = {}
index = 0
for i in rooms['Room']:
    vav_room[i] = [rooms['VAVmin'][index], rooms['VAVmax'][index], (rooms['VAVmin'][index] + rooms['VAVmax'][index])/2]
    index += 1

In [6]:
vav_room['CENTER 101']

[3500.0, 7000.0, 5250.0]

In [7]:
room_names = list(vav_room.keys())

demo run for debug purpose. 

**Build Website** 

In [11]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

server = app.server
activities = [{'label':'Lecture', 'value':'Lecture'}, {'label':'Studying', 'value':'Studying'}, {'label':'Singing', 'value':'Singing'}, {'label':'Social', 'value':'Social'}, {'label':'Exercising', 'value':'Exercising'}]
app.layout = html.Div([
    html.H6("Event Information"),
    #MAKE ROOM ID A DROP DOWN?
    html.Div(["RoomID: ",
              dcc.Dropdown(id='room-dropdown', options = [{'label':name, 'value':name} for name in room_names])]),
    html.Br(),
    html.Div(["VAV levels: ",
             dcc.Dropdown(id='vav-dropdown')]),
    html.Br(),
    html.Div(["Duration of Event (min): ",
              dcc.Input(id='time-input', value = 0, type='number')]),
    html.Br(),
    html.Div(["Number of Occupants: ",
              dcc.Input(id='occupant-input', value = 0, type='number')]),
    html.Br(),
    html.Div(["Activity: ",
              dcc.Dropdown(id='activity-dropdown', value ='test', options=activities)]),
#     html.Br(),
#     html.Button('Reset', id='reset-button'),
    html.Br(),
    html.Button('Go', id = 'go-button', n_clicks = 0),
    html.Br(),
    html.Div(id = 'calc-output', children = 'Enter values to calculate risk')

])

@app.callback(
    dash.dependencies.Output('vav-dropdown', 'options'),
    [dash.dependencies.Input('room-dropdown', 'value')]
)
def update_date_dropdown(name):
    return [{'label': 'cfm min', 'value': vav_room[name][0]}, {'label': 'current', 'value': vav_room[name][2]}, {'label': 'cfm max', 'value': vav_room[name][1]}]

@app.callback(
    dash.dependencies.Output('calc-output', 'children'),
    [dash.dependencies.Input('go-button', 'n_clicks')],
    [dash.dependencies.Input('activity-dropdown', 'value')],
    [dash.dependencies.Input('room-dropdown', 'value')],
    [dash.dependencies.Input('vav-dropdown', 'value')],
    [dash.dependencies.State('time-input', 'value')],
    [dash.dependencies.State('occupant-input', 'value')]
)
def update_calc(n_clicks, activity_dropdown, room_input,vav_dropdown, time_input, occupant_input):
    if n_clicks >= 1:
        comp_ir = ui_calc(activity_dropdown, room_input, time_input, occupant_input, rid_path,vav_dropdown)
        total_inf = int(occupant_input * comp_ir)
        to_return = 'The risk of an individual infected because of holding a(n) {} event for {} minutes in {} is {}%, given the most recent infection rates. With {} occupants, it is likely that {} occupant(s) will be infected.'.format(activity_dropdown, 
                                                                                                                                time_input, 
                                                                                                                                room_input, 
                                                                                                                                round((comp_ir * 100),2), 
                                                                                                                                occupant_input,
                                                                                                                                total_inf)
        return to_return
    else:
        return 'Enter Values to get risk calculation'

In [12]:
app.run_server(mode="inline")

---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
<ipython-input-11-62132710015a> in update_date_dropdown(name=None)
     35 )
     36 def update_date_dropdown(name):
---> 37     return [{'label': 'cfm min', 'value': vav_room[name][0]}, {'label': 'current', 'value': vav_room[name][2]}, {'label': 'cfm max', 'value': vav_room[name][1]}]
        global vav_room = {'CENTER 101': [3500.0, 7000.0, 5250.0], 'CENTER 115': [1290.0, 4300.0, 2795.0], 'CENTER 119': [1290.0, 4300.0, 2795.0], 'GH 242': [935.0, 2325.0, 1630.0], 'LEDDN AUD': [4000.0, 12000.0, 8000.0], 'PCYNH 106': [450.0, 1520.0, 985.0], 'PETER 110': [nan, nan, nan], 'TATA 3201': [1680.0, 3200.0, 2440.0], 'TATA 2101': [430.0, 960.0, 695.0], 'TATA2102': [430.0, 960.0, 695.0], 'York 3306': [870.0, 1300.0, 1085.0], 'York 3406': [1531.0, 1531.0, 1531.0], 'York 4406': [1500.0, 1500.0, 1500.0], 'TATA 4401': [nan, nan, nan], 'TATA 5600': [42